In [25]:
import nltk
sentences = [
    "Thank your message",
    "to show our words to the doctor,",
    "as his next contract checking, to all of us",
    "Anyway, I believe the team, although bit delay and less communication at recent days, they really tried best for paper and cooperation",
]
# simple context free grammar
grammar = nltk.CFG.fromstring("""
    S -> NP VP | VP | Adv S | S Conj S 
    NP -> Det N | Det N PP | Det Adj N | Det Adj N PP | NP PP | Pronoun | NP Conj NP | Adj NP | N | Det PP | N NP | Det Adj | N Adv
    PP -> P NP
    VP -> V NP | VP PP | V | Adv VP | VP Adv | V Pronoun | V VP | V S
    Det -> 'the' | 'a' | 'your' | 'our' | 'his' | 'all' | 'which' | 'some' | 'their'
    Adj -> 'next' | 'recent' | 'bit' | 'less' 
    Adv -> 'anyway' | 'really'  | 'even' | 'lately'  
    Pronoun -> 'us' | 'they' | 'i' | 'you' | 'he' | 'she' | 'who' | 'it' | 'them' | 'there'
    N -> 'doctor' | 'team' | 'message' | 'words' | 'contract' | 'days' | 'paper' | 'cooperation' | 'delay' | 'communication' | 'thoughts' | 'checking' | 'best' | 'delays'
    P -> 'to' | 'as' | 'for' | 'of' | 'at' | 'with' | 'in' | 'on' | 'by'
    V -> 'show' | 'believe' | 'tried' | 'thank' | 'helped' | 'assist' | 'convey' | 'will' | 'were' | 'did'
    Conj -> 'and' | 'though' | 'although'
""")
stop_words = [ '.', ',']
# create a parser
parser = nltk.ChartParser(grammar)
# parse the sentences
# fix the sentences
# to make them more grammatically correct
# and to match the grammar
def sentence_fixer(sentence):
    replace = {
        "your message": "you for your message,",
        "to show our words": "it helped convey our thoughts",
        "as his": "who will assist all of us with checking the",
        "checking, to all of us": ".",
        "anyway, ": "",
        ", although": " really did their best with the paper,",
        "bit delay": "though there were some delays",
        "at recent days,": "lately.",
        "they really tried best for paper and cooperation": ""
    }
    for key, value in replace.items():
        sentence = sentence.lower().replace(key, value)
    return sentence

sentences_fixed = [sentence_fixer(sentence) for sentence in sentences]

for sentence in sentences_fixed:
    print(f"Parsing: {sentence}")
    tokens = nltk.word_tokenize(sentence)
    tokens = [token.lower() for token in tokens if token not in stop_words]
    print(f"Tokens: {tokens}")
    
    try:
        tree = next(parser.parse(tokens))
        print(tree)
        tree.pretty_print()
    except StopIteration:
        print("No parse tree found.")
    print("\n")


Parsing: thank you for your message,
Tokens: ['thank', 'you', 'for', 'your', 'message']
(S
  (VP
    (V thank)
    (NP (NP (Pronoun you)) (PP (P for) (NP (Det your) (N message))))))
               S                      
               |                       
               VP                     
   ____________|___                    
  |                NP                 
  |       _________|___                
  |      |             PP             
  |      |      _______|____           
  |      NP    |            NP        
  |      |     |        ____|_____     
  V   Pronoun  P      Det         N   
  |      |     |       |          |    
thank   you   for     your     message



Parsing: it helped convey our thoughts to the doctor,
Tokens: ['it', 'helped', 'convey', 'our', 'thoughts', 'to', 'the', 'doctor']
(S
  (NP (Pronoun it))
  (VP
    (VP (V helped) (VP (V convey) (NP (Det our) (N thoughts))))
    (PP (P to) (NP (Det the) (N doctor)))))
                           S      

In [58]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

device = "cpu"

tokenizer = AutoTokenizer.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base")

model = AutoModelForSeq2SeqLM.from_pretrained("humarin/chatgpt_paraphraser_on_T5_base").to(device)

def paraphrase(
    question,
    num_beams=5,
    num_beam_groups=5,
    num_return_sequences=3,
    repetition_penalty=10.0,
    diversity_penalty=3.0,
    no_repeat_ngram_size=2,
    temperature=0.7,
    max_length=128
):
    input_ids = tokenizer(
        f'paraphrase: {question}',
        return_tensors="pt", padding="longest",
        max_length=max_length,
        truncation=True,
    ).input_ids.to(device)
    
    outputs = model.generate(
        input_ids, temperature=temperature, repetition_penalty=repetition_penalty,
        num_return_sequences=num_return_sequences, no_repeat_ngram_size=no_repeat_ngram_size,
        num_beams=num_beams, num_beam_groups=num_beam_groups,
        max_length=max_length, diversity_penalty=diversity_penalty
    )

    res = tokenizer.batch_decode(outputs, skip_special_tokens=True)

    return res

text1 = ["Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.",
"Hope you too, to enjoy it as my deepest wishes.",
"Thank your message to show our words to the doctor, as his next contract checking, to all of us.",
"I got this message to see the approved message.",
"In fact, I have received the message from the professor, to show me, this, a couple of days ago.",
"I am very appreciated the full support of the professor, for our Springer proceedings publication"
]

for text in text1:
    print(f"Original: {text}")
    paraphrased = paraphrase(text)
    for p in paraphrased:
        print(f"Paraphrased: {p}")
    print("\n")


Original: Today is our dragon boat festival, in our Chinese culture, to celebrate it with all safe and great in our lives.
Paraphrased: Our Chinese culture celebrates today's dragon boat festival to ensure the safety and prosperity of all who come from this world.
Paraphrased: In our Chinese culture, we celebrate today's dragon boat festival to make it a day to remember with great safety and comfort.
Paraphrased: According to Chinese customs, today's celebration of the dragon boat is a way to honor the safety and greatness of our lives.


Original: Hope you too, to enjoy it as my deepest wishes.
Paraphrased: Let's all have a wonderful time together, dear friends.
Paraphrased: May you also have a good time as my thoughts are with you.
Paraphrased: I hope you enjoy it as much as I do.


Original: Thank your message to show our words to the doctor, as his next contract checking, to all of us.
Paraphrased: As the doctor conducts his next contract check, I thank you for your message on how 

chatgpt_paraphraser:
  author={Vladimir Vorobev, Maxim Kuznetsov},
  title={A paraphrasing model based on ChatGPT paraphrases},
  year={2023}
